# 파이토치 CIFAR10 모델
- 코드 출처: https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

## modules import

- 현재 GPU 설정

## 전처리 설정

## 데이터 로드 및 확인

## 신경망 구성

## 손실함수와 옵티마이저

## 모델 학습

- 설명을 위해 `epochs=2`로 지정

## 모델의 저장 및 로드

## 모델 테스트

- 어떤 것을 잘 분류했고, 잘 못했는지 확인